<a href="https://colab.research.google.com/github/nshah-waripari/deep_learning/blob/main/pneumothrax_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Pneumothrax Classification with pretrained model**

In [ ]:
# pre-requisites:
# !pip install pretrainedmodels
# !pip install segmentation_models_pytorch
# NVIDIA AMP
# %%writefile setup.sh
# !git clone https://github.com/NVIDIA/apex
# !pip install -v --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex
!sh setup.sh

# mount the google drive
#from google.colab import drive
#drive.mount("/content/drive")
#!cp drive/My\ Drive/Colab\ Notebooks/pneumothrax_train_data/stage_2_train.csv sample_data/
#!cp drive/My\ Drive/Colab\ Notebooks/train/* sample_data/train/
#!cp drive/My\ Drive/Colab\ Notebooks/masks/* sample_data/mask/
#!unzip drive/My\ Drive/Colab\ Notebooks/masks.zip -d drive/My\ Drive/Colab\ Notebooks/masks/


setup.sh: 1: setup.sh: !git: not found
setup.sh: 2: setup.sh: !pip: not found
setup.sh: 3: setup.sh: !sh: not found


In [ ]:
#dataset loader
import torch

import numpy as np

from PIL import Image
from PIL import ImageFile

# sometimes, you will have images without an ending bit 
# this takes care of those kind of (corrupt) images Image
ImageFile.LOAD_TRUNCATED_IMAGES = True

class ClassificationDataSet:
    """
    A general classification dataset class that can be used for image classification tasks.
    For example: binary classification, multi-label classification etc.
    """
    def __init__(
        self,
        image_paths,
        targets,
        resize=None,
        augmentations=None
    ):
        """
        :param image_paths: list of path to images
        :param targets: numpy array
        :param resize: tuple, e.g. (256, 256), resizes image if not None
        :param augmentations: albumentation augmentations
        """
        self.image_paths = image_paths
        self.targets = targets
        self.resize = resize
        self.augmentations = augmentations

    def __len__(self):
        """
        Return the total number of samples in the dataset
        """
        return len(self.image_paths)

    def __getitem__(self, item):
        """
        For a given "item" index, return everything we need to
        train a given model
        """
        # Use PIL to open the image
        image = Image.open(self.image_paths[item])
        # convert image to RGB, we have single channel images
        image = image.convert("RGB")
        # grab correct targets
        targets = self.targets[item]

        # resize if needed
        if self.resize is not None:
            image = image.resize(
                (self.resize[1], self.resize[0]),
                resample = Image.BILINEAR
            )
        # convert image to numpy array
        image = np.array(image)

        # if we have albumentation augmentations
        # add item to the image
        if self.augmentations is not None:
            augmented = self.augmentations(image=image)
            image = augmented["image"]
        # pytorch expected CHW instead HWC
        image = np.transpose(image, (2,0,1)).astype(np.float32)

        # return tensors of image and targets       
        return {
            "images": torch.tensor(image, dtype=torch.float),
            "targets": torch.tensor(targets, dtype=torch.long)
        }




In [ ]:
# engine
import torch
import torch.nn as nn

from tqdm import tqdm

def train(data_loader, model, optimizer, device):
    """
    This function does the training for one epoch
    :param data_loader: pytorch dataloader
    :param model: pytorch model
    :device: cuda/cpu
    """

    # put the model in training mode
    model.train()

    # iterate over every batch of data in the dataloader
    for data in data_loader:
        # extract "image" and "targets" from dataset class
        inputs = data["images"]
        targets = data["targets"]

        # move the inputs and targets to cpu/cuda device
        inputs = inputs.to(device, dtype=torch.float)
        targets = targets.to(device, dtype=torch.float)

        # zero grad the optimizer
        optimizer.zero_grad()
        # do the forward step
        outputs = model(inputs)

        # calculate the loss
        loss = nn.BCEWithLogitsLoss()(outputs, targets.view(-1, 1))
        # do the back propagation
        loss.backward()
        # step optimizer
        optimizer.step()

def evaluate(data_loader, model, device):
    """
    This function does the evaluation per epoch
    :param data_loader: pytorch dataloader
    :param model: pytorch model
    :param device: cpu/cuda device    
    """
    # put the model in evaluation mode
    model.eval()

    # init lists to store targets and outputs
    final_targets = []
    final_outputs = []

    with torch.no_grad():
        for data in data_loader:
            inputs = data["images"]
            targets = data["targets"]

            # move the inputs and targets to cpu/cuda device
            inputs = inputs.to(device, dtype=torch.float)
            targets = targets.to(device, dtype=torch.float)

            # do the forward step
            output = model(inputs)

            # convert the targets and outputs to lists
            targets = targets.detach().cpu().numpy().tolist()
            output = output.detach().cpu().numpy().tolist()

            # extend the original list
            final_targets.extend(targets)
            final_outputs.extend(output)

    # return final outputs and targets
    return final_outputs, final_targets



In [ ]:
# model
import torch.nn as nn
import pretrainedmodels

def get_model_alexnet(pretrained):
    if pretrained:
        model = pretrainedmodels.__dict__["alexnet"](
            pretrained = 'imagenet'
        )
    else:
        model = pretrainedmodels.__dict__["alexnet"](
            pretrained = None
        )        
    # add sequential layer to the model
    model.last_linear = nn.Sequential(
        nn.BatchNorm1d(4096),
        nn.Dropout(p=0.25),
        nn.Linear(in_features=4096, out_features=2048),
        nn.ReLU(),
        nn.BatchNorm1d(2048, eps=1e-05, momentum=0.1),
        nn.Dropout(p=0.5),
        nn.Linear(in_features=2048, out_features=1)

    )
    return model

def get_model_resnet(pretrained):
    if pretrained:
        model = pretrainedmodels.__dict__["resnet18"](
            pretrained = 'imagenet'
        )
    else:
        model = pretrainedmodels.__dict__["resnet18"](
            pretrained = None
        )        
    # add sequential layer to the model
    model.last_linear = nn.Sequential(
        nn.BatchNorm1d(512),
        nn.Dropout(p=0.25),
        nn.Linear(in_features=512, out_features=2048),
        nn.ReLU(),
        nn.BatchNorm1d(2048, eps=1e-05, momentum=0.1),
        nn.Dropout(p=0.5),
        nn.Linear(in_features=2048, out_features=1)

    )
    print(model)
    return model



In [ ]:
# train
import joblib
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import tree
import os
import argparse
import albumentations
import torch

# location of train.csv and image files
data_path = 'drive/My Drive/Colab Notebooks/'

# device name
device = "cuda"

# number of epochs
epochs = 10

# load the dataframe
df = pd.read_csv(os.path.join(data_path,"pneumothrax_train_data/stage_2_train.csv"))
# df = pd.read_csv(os.path.join(data_path,"stage_2_train.csv"))

#fetch all images with ImageId
images = df.ImageId.values.tolist()

# creat a list of image locations
images = [
    os.path.join(data_path,"train", i + ".png") for i in images
]
print(images[0])
targets = [0. if item=="-1" else 1. for item in df.EncodedPixels.values]
print(targets[:50])
# move the model to device
model = get_model_resnet(pretrained=True)
model.to(device)

# mean and std values of RGB channels for imagenet dataset 
# we use these pre-calculated values when we use weights # from imagenet.    
mean = (0.485, 0.456, 0.406)
std = (0.229, 0.224, 0.225)

# albumentation is an image augmentation library
# that allows you to do diffrerent type of augmentations
# simple normalization in our case
aug = albumentations.Compose(
    [
        albumentations.Normalize(
        mean, std, max_pixel_value=255.0, always_apply=True
        )
    ]
)

# simple train/valid split
train_images, valid_images, train_targets, valid_targets = train_test_split(images, targets, stratify=targets, random_state=42)

# fetch classification dataset
train_dataset = ClassificationDataSet(train_images,
  train_targets,
  resize=(227, 227),
  augmentations=aug
  )

  # create batches of data from
  # classificatin dataset class using torch dataloader
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size = 16, shuffle=True, num_workers = 4
)



# same for validation 
valid_dataset = ClassificationDataSet(valid_images,
  valid_targets,
  resize=(227, 227),
  augmentations=aug
  )

valid_loader = torch.utils.data.DataLoader(
    valid_dataset, batch_size = 16, shuffle=True, num_workers = 4
)

# simple adam optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=5e-4)

# train and print auc score for all epochs
for epoch in range(epochs):
    print(f"Training Started for epoch: {epoch}")
    train(train_loader, model, optimizer, device)
    predictions, valid_targets = evaluate(valid_loader, model, device)
    roc_auc = metrics.roc_auc_score(valid_targets, predictions)
    print(
        f"Epoch={epoch}, Valid ROC AUC = {roc_auc}"
    )


drive/My Drive/Colab Notebooks/train/1.2.276.0.7230010.3.1.4.8323329.3678.1517875178.953520.png
[1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 0.0, 0.0]
ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=F

In [ ]:
torch.save(model.state_dict(), 'drive/My Drive/Colab Notebooks/pneumothrax_model_output/pneumothrax_resnet18.pth')

In [ ]:
#!ls drive/My\ Drive/Colab\ Notebooks/pneumothrax_train_data/
#!unzip drive/My\ Drive/Colab\ Notebooks/pneumothrax_train_data/train.zip -d drive/My\ Drive/Colab\ Notebooks/pneumothrax_train_data/

**Segmentation**

In [ ]:
# SIIM Dataset
import os
import glob
import torch

import numpy as np
import pandas as pd

from PIL import Image, ImageFile

from tqdm import tqdm
from collections import defaultdict
from torchvision import transforms

from albumentations import (
    Compose,
    OneOf,
    RandomBrightnessContrast,
    RandomGamma,
    ShiftScaleRotate,
)

ImageFile.LOAD_TRUNCATED_IMAGES = True
TRAIN_PATH = 'drive/My Drive/Colab Notebooks/train'
TRAIN_MASK_PATH = 'drive/My Drive/Colab Notebooks/masks'

class SIIMDataset(torch.utils.data.Dataset):
  def __init__(
      self,
      image_ids,
      transform = True,
      preprocessing_fn = None
  ):

    """
    Dataset class for segmentation problem
    :param iamge_ids: ids of the images as a list
    :param transform: Boolean, no transformation in validation dataset
    :param  preprocessing_fn: Image preprocessing function
    """

    # create a empty dictionary to store image
    # and mask paths
    self.data = defaultdict(dict)

    # for augmenataions
    self.transform = transform
    
    # for image preprocessing
    self.preprocessing_fn = preprocessing_fn

    # albumenation augmentations
    # shift, scale and rotate
    # with 80% probability
    # one of gamma and brightness/contrast
    self.aug = Compose(
        [
        ShiftScaleRotate(
            shift_limit=0.0625,
            scale_limit=0.1,
            rotate_limit=10,
            p=0.8
        ),
        OneOf(
            [
              RandomGamma(gamma_limit = (90, 110)),
              RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1),
            ],
            p=0.5,
        ),
        ]
    )

    # iterate over image_ids to store
    # image and mask paths
    for counter, imgid in enumerate(image_ids):
      files = glob.glob(os.path.join(TRAIN_PATH, imgid, "*.png"))
      self.data[counter] = {
          "img_path":os.path.join(TRAIN_PATH, imgid + ".png"),
          "mask_path":os.path.join(TRAIN_MASK_PATH, imgid + ".png"),
      }
  
  def __len__(self):
    return len(self.data)
  
  def __getitem__(self, item):
    # for a given item
    # return image and mask tensors
    img_path = self.data[item]["img_path"]
    mask_path = self.data[item]["mask_path"]

    # read image convert to RGB
    img = Image.open(img_path)
    img = img.convert("RGB")

    # to numpy array
    img = np.array(img)

    # read mask image
    mask = Image.open(mask_path)
    # to numpy array
    mask = np.array(mask)

    # convert to binary float matrix
    mask = (mask >= 1).astype("float32")

    # apply transforms (to training data only)
    if self.transform is True:
      augmented = self.aug(image=img, mask=mask)
      img = augmented["image"]
      mask = augmented["mask"]

    # preproces the image using 
    # supplied preprocessing function
    # img = self.preprocessing_fn(img)

    # return image and mask tensors
    return {
        "image":transforms.ToTensor()(img),
        "mask":transforms.ToTensor()(mask).float(),
    }



In [ ]:
# train
import os
import sys
import torch

import numpy as np
import pandas as pd
import segmentation_models_pytorch as smp
import torch.nn as nn
import torch.optim as optim

from apex import amp
from collections import OrderedDict
from sklearn import model_selection
from sklearn import metrics
from tqdm import tqdm
from torch.optim import lr_scheduler

TRAINING_CSV ="drive/My Drive/Colab Notebooks/pneumothrax_train_data/stage_2_train.csv"
TRAINING_BATCH_SIZE = 8
TEST_BATCH_SIZE = 4
EPOCHS = 10

# define the encoder for UNET
ENCODER = "resnet18"

# use imagenet pretrained weights for encoder
ENCODER_WEIGHTS = "imagenet"

DEVICE ="cuda"

def train(dataset, data_loader, model, criterion, optimizer):
  """
  training function that trains for one epoch
  :param dataset: dataset class (SIIMDataset)
  :param data_loader: torch dataset loader :param model: model
  :param criterion: loss function
  :param optimizer: adam, sgd, etc.
  """
  # put the model in train mode
  model.train()

  # calculate the number of batched
  num_batches = int(len(dataset)/data_loader.batch_size)

  # init tqdm to track progress
  tk0 = tqdm(data_loader, total=num_batches)
  # loop over all batches
  for d in tk0:
    # fetch input
    # mask and images from dataset batch
    inputs = d["image"]
    targets = d["mask"]

    # move images and masks to device
    inputs = inputs.to(DEVICE, dtype=torch.float)
    targets = targets.to(DEVICE, dtype=torch.float)

    # zero grad the optimizer
    optimizer.zero_grad()

    # forward step the model
    outputs = model(inputs)

    # calculate the loss
    loss = criterion(outputs, targets)

    # backward loss is calculated on a scaled loss
    # context since we are using mixed precision training
    # if you are not using mixed precision training,
    # you can use loss.backward() and delete the following
    # two lines of code
    with amp.scale_loss(loss, optimizer) as scaled_loss:
      scaled_loss.backward()
    # step the optimizer
    optimizer.step()

  # close tqdm
  tk0.close()

def evaluate(dataset, data_loader, model):
  """
  evaluation function to calculate loss on validation
  set for one epoch
  :param dataset: dataset class (SIIMDataset)
  :param data_loader: torch dataset loader
  :param model: pytorch model
  """

  # put the model in evaluation mode
  model.eval()

  # init final loss to zero
  final_loss = 0

  # calculate the number of batched
  num_batches = int(len(dataset)/data_loader.batch_size)

  # list of valid targets and outputs
  final_outputs = []
  final_targets = []

  # init tqdm to track progress
  tk0 = tqdm(data_loader, total=num_batches)
  # loop over all batches
  with torch.no_grad():
    for d in tk0:
      # fetch input
      # mask and images from dataset batch
      inputs = d["image"]
      targets = d["mask"]
      inputs = inputs.to(DEVICE, dtype=torch.float)
      targets = targets.to(DEVICE, dtype=torch.float)
      output = model(inputs)
      loss = criterion(output, targets)
      # add to final loss
      final_loss += loss

      # convert the targets and outputs to lists
      targets = targets.detach().cpu().numpy().tolist()
      output = output.detach().cpu().numpy().tolist()

      # extend the original list
      final_targets.extend(targets)
      final_outputs.extend(output)

  tk0.close()

  # return the average loss over all batches along with predictions and targets
  return final_loss/num_batches, final_outputs, final_targets

if __name__=="__main__":
  # read the training csv file
  df = pd.read_csv(TRAINING_CSV)

  # split the data into training and validation set
  df_train, df_valid = model_selection.train_test_split(df, random_state=42, test_size=0.1)

  # training and validation images list
  training_images = df_train.ImageId.values
  validation_images = df_valid.ImageId.values

  # fetch unet model from segmentation models
  # with specified encoder architecture
  model = smp.Unet(
      encoder_name = ENCODER,
      encoder_weights = ENCODER_WEIGHTS,
      classes = 1,
      activation = None,

  )

  # segmentation model provides you with a preprocessing
  # function that can be used for normalizing images
  # normalization is only applied on images and not masks
  prep_fn = smp.encoders.get_preprocessing_fn(
      ENCODER,
      ENCODER_WEIGHTS
  )
  
  model.to(DEVICE)

  # init training dataset
  # transfor is true for training
  train_dataset = SIIMDataset(
      training_images,
      transform = True,
      preprocessing_fn = prep_fn
  )

  # wrap training dataset in torch's loader
  train_loader = torch.utils.data.DataLoader(
      train_dataset,
      batch_size = TRAINING_BATCH_SIZE,
      shuffle = True,
      num_workers = 12
  )

  # init validation dataset
  # transfor is false for training
  valid_dataset = SIIMDataset(
      validation_images,
      transform = False,
      preprocessing_fn = prep_fn
  )

  # wrap validation dataset in torch's loader
  valid_loader = torch.utils.data.DataLoader(
      valid_dataset,
      batch_size = TEST_BATCH_SIZE,
      shuffle = True,
      num_workers = 4
  )

  # define criterion
  criterion = nn.BCEWithLogitsLoss()

  # use adam optimizer
  optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
  # reduce learning rate when we reach plateau on loss
  scheduler = lr_scheduler.ReduceLROnPlateau(
      optimizer, mode="min", patience=3, verbose=True
  )

  # wrap model and optimizer with NVIDIA's apex
  # this is used for mixed precision training
  # if you have a GPU that supports mixed precision,
  # this is very helpful as it will allow us to fit larger images
  # and larger batches
  mode, optimizer = amp.initialize(
      model, optimizer, opt_level="O1", verbosity=0
  )

  # if more then one GPU
  if torch.cuda.device_count() > 1:
    print("Using {torch.cuda.device_count()} GPUS!")
    model = nn.DataParallel(model)

  # some logging
  print(f"Training batch size: {TRAINING_BATCH_SIZE}")
  print(f"Test batch size: {TEST_BATCH_SIZE}")
  print(f"Epochs: {EPOCHS}")
  print(f"Number of training images: {len(train_dataset)}")
  print(f"Number of validation images: {len(valid_dataset)}")
  print(f"Encoder: {ENCODER}")

  # loop over all epochs
  for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch}\n")
    # train for one epoch
    train(
        train_dataset,
        train_loader,
        model,
        criterion,
        optimizer
    )
    print(f"Validation Epoch: {epoch}")
    # calculate validation loss
    val_log, final_outputs, final_targets = evaluate(
      valid_dataset,
      valid_loader,
      model
    )
    # final_outputs = sum(final_outputs, [])
    # final_outputs = sum(final_outputs, [])
    # final_targets = sum(final_targets, [])
    # final_targets = sum(final_targets, [])
    # final_targets[0]
    # print(final_targets[0])
    # print(final_outputs[0])
    # roc_auc = metrics.roc_auc_score(final_outputs, final_targets)
    accuracy = (np.array(final_outputs)==np.array(final_targets)).mean()    
    print(
      f"Epoch={epoch}, Validation Accuracy = {accuracy}, Final Loss = {val_log.item()}"
    )

    # print(f"Validation loss: {val_log.item()}\n")
    scheduler.step(val_log.item())
    print("\n") 






  0%|          | 0/1457 [00:00<?, ?it/s]

Training batch size: 8
Test batch size: 4
Epochs: 10
Number of training images: 11658
Number of validation images: 1296
Encoder: resnet18
Training epoch: 0



1458it [02:18, 10.54it/s]
  0%|          | 0/324 [00:00<?, ?it/s]

Validation Epoch: 0


  0%|          | 0/1457 [00:00<?, ?it/s]

Epoch=0, Validation Accuracy = 0.0, Final Loss = 0.018228191882371902


Training epoch: 1



1458it [02:31,  9.64it/s]
  0%|          | 0/324 [00:00<?, ?it/s]

Validation Epoch: 1


 65%|██████▍   | 209/324 [00:17<00:31,  3.64it/s]

RuntimeError: ignored

In [ ]:
torch.save(model.state_dict(), 'drive/My Drive/Colab Notebooks/pneumothrax_model_output/pneumothrax_unet_resnet18.pth')

In [ ]:
del model